
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://cct.ued.vnu.edu.vn/admin//lienketlinkphai/brtunv-7_87_anhlienket.png" alt="Databricks Learning" style="width: 400px">
</div>


**Import thư viện**

In [0]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.SparkSession
import spark.implicits._

**Chạy phần cell dưới đây để nạp dữ liệu vào notebook (Chỉ cần chạy, không cần sửa gì ở đây)**

In [0]:
dbutils.widgets.removeAll

In [0]:
%python
import requests
import os
from pyspark.sql import SparkSession

def list_files_in_github_folder(repo_owner, repo_name, folder_path, branch="main"):
    api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{folder_path}?ref={branch}"
    response = requests.get(api_url)
    
    if response.status_code == 200:
        contents = response.json()
        return [item['path'] for item in contents if item['type'] == 'file']
    else:
        print(f"Failed to list files in {folder_path}, Status Code: {response.status_code}")
        return []

def download_github_file(repo_owner, repo_name, file_path, save_path, branch="main"):
    raw_url = f"https://raw.githubusercontent.com/{repo_owner}/{repo_name}/{branch}/{file_path}"
    response = requests.get(raw_url)

    if response.status_code == 200:
        dbfs_path = f"/dbfs{save_path}"
        os.makedirs(os.path.dirname(dbfs_path), exist_ok=True)
        with open(dbfs_path, "wb") as file:
            file.write(response.content)
        print(f"Downloaded: {file_path} to {save_path}")
    else:
        print(f"Failed to download {file_path}, Status Code: {response.status_code}")


# Configuration
repo_owner = "bamaga12"
repo_name = "university-databricks-trainning"
github_folder = "data/events/events-2020-07-03"
dbfs_base_path = "/mnt/github_files/" + github_folder

# Get list of files and download them
file_list = list_files_in_github_folder(repo_owner, repo_name, github_folder)

for file_path in file_list:
    save_path = f"{dbfs_base_path}/{os.path.basename(file_path)}"
    download_github_file(repo_owner, repo_name, file_path, save_path)

dbutils.widgets.text("events", f"file:///dbfs{dbfs_base_path}")

**Đọc dữ liệu**


# Bài 1
Hãy tạo DataFrame `input` bằng cách đọc file `json` với `schema` và `path` đã cho như dưới đây:

In [0]:
val schema = "device STRING, ecommerce STRUCT<purchase_revenue_in_usd: DOUBLE, total_item_quantity: BIGINT, unique_items: BIGINT>, event_name STRING, event_previous_timestamp BIGINT, event_timestamp BIGINT, geo STRUCT<city: STRING, state: STRING>, items ARRAY<STRUCT<coupon: STRING, item_id: STRING, item_name: STRING, item_revenue_in_usd: DOUBLE, price_in_usd: DOUBLE, quantity: BIGINT>>, traffic_source STRING, user_first_touch_timestamp BIGINT, user_id STRING"

val path = dbutils.widgets.get("events")

In [0]:
val input = // TO-DO


# Bài 2
Hãy cho biết trung bình doanh thu kiếm được trung bình từ `ecommerce.purchase_revenue_in_usd` của mỗi `user_id` là bao nhiêu từ DataFrame `input`

#### YÊU CẦU ĐẦU RA:

• Kết quả chứa thông tin như sau:
`user_id`, `avg_purchase_revenue` và sắp xếp theo `avg_purchase_revenue` theo thứ tự giảm dần

| Tên cột | Kiểu dữ liệu |
|------------|-----------|
| user_id    | `String`      | 
| avg_purchase_revenue    | `Double`      |

• Dữ liệu sẽ được lưu tại đường dẫn: **`file:///dbfs/ThucHanhBuoi8/Bai2`** với định dạng file `parquet` và phương thức nén `snappy`. Trong đó, số lượng file mong muốn tại đường dẫn là 5 file.

#### KẾT QUẢ MẪU:

| `user_id` | `avg_purchase_revenue` |
|------------|-----------|
|`UA000000106983464` |	`5030`|
|`UA000000107159687` |	`4475` |
| `UA000000107208167`	| `4394`|

In [0]:
val lesson2 = //TO-DO


# Bài 3
Hãy cho biết tỷ lệ mua hàng trên mỗi loại thiết bị `device`

#### YÊU CẦU ĐẦU RA:

• Kết quả chứa thông tin như sau:
`device`, `total_events`, `purchase_events`,`conversion_rate` với `purchase_events` được lấy từ những thao tác có `ecommerce.purchase_revenue_in_usd` lớn hơn 0 trên tổng số thao tác của mỗi nền tảng. `conversion_rate` chính là tỷ lệ `purchase_events` trên tổng `total_events`.

• Làm tròn `conversion_rate` 5 số sau dấu phẩy.

| Tên cột | Kiểu dữ liệu |
|------------|-----------|
| device    | `String`      | 
| total_events | `Long` |
| purchase_events | `Long`|
| conversion_rate    | `Double`      |

• Dữ liệu sẽ được lưu tại đường dẫn: **`file:///dbfs/ThucHanhBuoi8/Bai3`** với định dạng file `csv` và phương thức nén `gzip`. Tạo ra các sub-folder tương ứng với mỗi `device` duy nhất.

#### KẾT QUẢ MẪU:

| `device` | `total_events` | `purchase_events` | `conversion_rate` |
|------------|-----------| -| -|
|`Chrome OS` |	`49197`| `933` | `0.01896`
|`Windows` |	`110872` | `2073` | `0.01870`
| `iOS`	| `109423`| `2023` | `0.01849`

In [0]:
val lesson3 = //TO-DO


# Bài 4
Sử dụng kết quả của bài 2, Hãy thử lưu dữ liệu thành 100 file khác nhau. Trong đó với mỗi file sẽ chỉ lưu những `user_id` có 2 số cuối giống nhau.


#### Ví dụ:

• `user_id` = `UA000000106983464` và `UA000000106986464` sẽ lưu cùng vào 1 file do 2 chữ số cuối cùng chứa số `64` 


• Dữ liệu sẽ được lưu tại đường dẫn: **`file:///dbfs/ThucHanhBuoi8/Bai4`** với định dạng file `json`

#### KẾT QUẢ MẪU:

| `user_id` | `avg_purchase_revenue` |
|------------|-----------|
|`UA000000106983464` |	`5030`|
|`UA000000107159687` |	`4475` |
| `UA000000107208167`	| `4394`|

In [0]:
val lesson4 = //TO-DO